In [ ]:
!pip install cohere
!pip install annoy

In [ ]:
import cohere
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from annoy import AnnoyIndex
import pandas as pd

In [ ]:
question = "Which Sydney beach should I visit?"



text = """

Sydney is world famous for beautiful beaches. These beaches offer different vibes and attractions, from bustling crowds and great surfing conditions to more tranquil and family-friendly environments.



Bondi Beach: Bondi is perhaps the most famous beach in Sydney, if not Australia. It's known for its golden sands, vibrant atmosphere, and excellent surfing conditions. The Bondi to Coogee coastal walk offers stunning views of the coastline.



Manly Beach: Easily accessible by a scenic ferry ride from Circular Quay, Manly Beach is known for its laid-back atmosphere and family-friendly environment. The beach is great for swimming, surfing, and various water sports.



Cronulla Beach: Located in the Sutherland Shire, Cronulla offers a more relaxed atmosphere compared to some of the busier city beaches. It's a great spot for swimming, picnicking, and enjoying a range of seaside cafes and restaurants.




Bronte Beach: Situated between Bondi and Coogee, Bronte Beach is popular among both locals and visitors. It's a smaller, quieter beach with a beautiful park area and a natural rock pool that's ideal for swimming.



Tamarama Beach: Also known as "Glamarama" due to its popularity among the fashion-conscious crowd, Tamarama Beach is a smaller and more secluded option. It's surrounded by rocky cliffs and offers strong surf, making it a favorite among experienced surfers.

"""

In [ ]:
#Split into a list of paragraphs.
texts=text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts=np.array([t.strip(' \n') for t in texts if t])


In [ ]:
from config import API_KEY

In [ ]:
co=cohere.Client(API_KEY)

In [ ]:
# Get the Embeddings
response=co.embed(
    texts =texts.tolist(),
).embeddings

In [ ]:
texts

array(['Sydney is world famous for beautiful beaches. These beaches offer different vibes and attractions, from bustling crowds and great surfing conditions to more tranquil and family-friendly environments.',
       "Bondi Beach: Bondi is perhaps the most famous beach in Sydney, if not Australia. It's known for its golden sands, vibrant atmosphere, and excellent surfing conditions. The Bondi to Coogee coastal walk offers stunning views of the coastline.",
       'Manly Beach: Easily accessible by a scenic ferry ride from Circular Quay, Manly Beach is known for its laid-back atmosphere and family-friendly environment. The beach is great for swimming, surfing, and various water sports.',
       "Cronulla Beach: Located in the Sutherland Shire, Cronulla offers a more relaxed atmosphere compared to some of the busier city beaches. It's a great spot for swimming, picnicking, and enjoying a range of seaside cafes and restaurants.",
       "Bronte Beach: Situated between Bondi and Coogee, Br

In [ ]:
(np.array(response).shape)

(6, 4096)

In [ ]:
#Check the dimension of embeddings
embeds=np.array(response)

# Create the search index, pass the size of embeddings
search_index=AnnoyIndex(embeds.shape[1],'angular')

#Add all the vectors to the search index
for i in range(len(embeds)):
  search_index.add_item(i,embeds[i])

search_index.build(10)  #10 trees
search_index.save('test.ann')

True

In [ ]:
def search_text(query):
  #Get the query's embeddings
  query_embed = co.embed(texts=[query]).embeddings

  #Retrieve the nearest neighbors
  similar_item_ids=search_index.get_nns_by_vector(query_embed[0],
                                                  10,
                                                  include_distances=True)
  search_results = texts[similar_item_ids[0]]
  return search_results

In [ ]:
results= search_text(question)

print(results[0])

Bondi Beach: Bondi is perhaps the most famous beach in Sydney, if not Australia. It's known for its golden sands, vibrant atmosphere, and excellent surfing conditions. The Bondi to Coogee coastal walk offers stunning views of the coastline.


In [ ]:
def ask_llm(question, num_generations=1):
    # Search the text archive
    results = search_text(question)
    # Get the top result
    context = results[0]
    # Prepare the prompt
    prompt = f"""
    More information about Australian beaches at australia.com:
    {context}
    Question: {question}
     Extract the answer of the question from the text provided.
     If the text doesn't contain the answer,
     reply that the answer is not available."""

    prediction = co.generate(
         prompt=prompt,
         max_tokens=70,
         model="command-nightly",
         temperature=0.5,
         num_generations=num_generations
         )



    return prediction.generations



In [ ]:
results = ask_llm(question,)

print(results[0])

id='3c430ee1-7c52-4896-8fff-dfe7f7362b79' text='Bondi Beach.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'


In [ ]:
question = "Which Sydney beach is for family?"
results = ask_llm(question,)
print(results[0])

id='ae7a4055-129b-4e3c-995e-782c62e23957' text='The answer is not available.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'


In [ ]:
question = "Which Sydney beach does has a rock pool?"
results = ask_llm(question,)
print(results[0])

id='01cf956d-2910-4cfa-9b7e-5e9d977648b9' text='Bronte Beach.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'


In [ ]:
question = "What is Sydney famous for?"
results = ask_llm(question,)
print(results[0])

id='56d852bd-8171-4402-9f60-5b8b1c3c0f91' text='Sydney is famous for beautiful beaches.' index=None likelihood=None token_likelihoods=None finish_reason='COMPLETE'
